In [1]:
!pip install torch torchtext transformers sentencepiece pandas tqdm datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time

In [3]:
data_sample = load_dataset("QuyenAnhDE/Diseases_Symptoms")

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [4]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [5]:
updated_data = [{'Name': item['Name'], 'Symptoms':item['Symptoms']} for item in data_sample['train']]

In [6]:
df = pd.DataFrame(updated_data)
df.head()

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [7]:
df['Symptoms'] = df['Symptoms'].apply(lambda x: ', '.join(x.split(', ')))

In [8]:
df['Symptoms']

0      Palpitations, Sweating, Trembling, Shortness o...
1               Hoarseness, Vocal Changes, Vocal Fatigue
2      Short stature, Gonadal dysgenesis, Webbed neck...
3      Absence or undescended testicle(s), empty scro...
4      Nausea, vomiting, abdominal pain, General mala...
                             ...                        
395    Severe abdominal or back pain, blood in urine,...
396    Fragile bones, loss of height over time, back ...
397    Joint pain, stiffness, swelling, fatigue, loss...
398    Frequent urination, Increased thirst, Weight loss
399    Fatigue, Increased hunger, Slow healing of wounds
Name: Symptoms, Length: 400, dtype: object

In [9]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

In [10]:
if torch.backends.mps.is_available():
  device = torch.device('mps')
else:
  print("MPS is not available. Using CPU instead.")
  device = torch.device('cpu')

MPS is not available. Using CPU instead.


In [11]:
device

device(type='cpu')

In [12]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [13]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [14]:
BATCH_SIZE = 8
df.describe()

,Name,Symptoms
count,400,400
unique,392,395
top,Sciatica,"Swelling, pain, dry mouth, bad taste"
freq,3,3


In [15]:
class LanguageDataset(Dataset):
  def __init__(self, df, tokenizer):
    self.labels = df.columns
    self.data = df.to_dict(orient='records')
    self.tokenizer = tokenizer
    x = self.fittest_max_length(df)
    self.max_length = x

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    x = self.data[idx][self.labels[0]]
    y = self.data[idx][self.labels[1]]
    text = f"{x} | {y}"
    token = self.tokenizer.encode_plus(text, max_length=128, pad_to_max_length=True,truncation=True ,return_tensors='pt')
    return token

  def fittest_max_length(self, df):
    max_length = max(len(max(df[self.labels[0]], key=len)), len(max(df[self.labels[1]],key=len)))
    x = 2
    while x < max_length: x = x*2
    return x

In [16]:
data_samples = LanguageDataset(df, tokenizer)

In [17]:
data_samples

In [18]:
train_size = int(0.8 * len(data_samples))
val_size = len(data_samples) - train_size
train_data, test_data = random_split(data_samples, [train_size, val_size])

In [19]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE)
val_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [20]:
num_epochs = 5
batch_size = BATCH_SIZE
model_name = 'distilgpt2'
gpu = 0

In [21]:
criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token

In [22]:
results = pd.DataFrame(columns=['epoch', 'transformer', 'batch_size', 'gpu', 'training_loss', 'validation_loss', 'epoch_duration_sec'])

In [23]:
for epoch in range(num_epochs):
  start_time = time.time()
  model.train()
  epoch_training_loss = 0
  train_iterator = tqdm(train_loader, desc=f"Training epoch{epoch+1}/{num_epochs}, Batch_size:{batch_size}, transformer:{model_name}")
  for batch in train_iterator:
    optimizer.zero_grad()
    inputs = batch['input_ids'].squeeze(1).to(device)
    targets = inputs.clone()
    outputs = model(input_ids = inputs, labels=targets)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    train_iterator.set_postfix({'Training loss': loss.item()})
    epoch_training_loss += loss.item()
  avg_training_loss = epoch_training_loss / len(train_loader)

  # validation
  model.eval()
  epoch_validation_loss = 0
  total_loss = 0
  valid_iterator = tqdm(val_loader, desc=f"Validation epoch{epoch+1}/{num_epochs}")
  with torch.no_grad():
    for batch in valid_iterator:
      inputs = batch['input_ids'].squeeze(1).to(device)
      targets = inputs.clone()
      outputs = model(input_ids = inputs, labels=targets)
      loss = outputs.loss
      epoch_validation_loss += loss.item()
      total_loss += loss
      train_iterator.set_postfix({'Validation loss': loss.item()})
    avg_epoch_validation_loss = epoch_validation_loss / len(valid_iterator)

    end_time = time.time()
    epoch_duration_sec = end_time - start_time
    new_row ={
        'transformer' : model_name,
        'batch_size' : batch_size,
        'gpu' : gpu,
        'training_loss' : avg_training_loss,
        'validation_loss' : avg_epoch_validation_loss,
        'epoch_duration_sec' : epoch_duration_sec
    }
    results.loc[len(results)] = new_row
    print(f"Epoch: {epoch+1}, Validation Loss: {total_loss/len(val_loader)}")


Training epoch1/5, Batch_size:8, transformer:distilgpt2:   0%|          | 0/40 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Training epoch1/5, Batch_size:8, transformer:distilgpt2: 100%|██████████| 40/40 [08:11<00:00, 12.29s/it, Training loss=0.828]
Validation epoch1/5: 100%|██████████| 10/10 [00:37<00:00,  3.72s/it]


Epoch: 1, Validation Loss: 0.6792088150978088


Training epoch2/5, Batch_size:8, transformer:distilgpt2:   0%|          | 0/40 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Training epoch2/5, Batch_size:8, transformer:distilgpt2: 100%|██████████| 40/40 [08:06<00:00, 12.17s/it, Training loss=0.575]
Validation epoch2/5: 100%|██████████| 10/10 [00:44<00:00,  4.43s/it]


Epoch: 2, Validation Loss: 0.6388952732086182


Training epoch3/5, Batch_size:8, transformer:distilgpt2:   0%|          | 0/40 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Training epoch3/5, Batch_size:8, transformer:distilgpt2: 100%|██████████| 40/40 [08:03<00:00, 12.09s/it, Training loss=0.469]
Validation epoch3/5: 100%|██████████| 10/10 [00:36<00:00,  3.66s/it]


Epoch: 3, Validation Loss: 0.6909344792366028


Training epoch4/5, Batch_size:8, transformer:distilgpt2:   0%|          | 0/40 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Training epoch4/5, Batch_size:8, transformer:distilgpt2: 100%|██████████| 40/40 [07:52<00:00, 11.82s/it, Training loss=0.323]
Validation epoch4/5: 100%|██████████| 10/10 [00:38<00:00,  3.84s/it]


Epoch: 4, Validation Loss: 0.7638903856277466


Training epoch5/5, Batch_size:8, transformer:distilgpt2:   0%|          | 0/40 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Training epoch5/5, Batch_size:8, transformer:distilgpt2: 100%|██████████| 40/40 [08:02<00:00, 12.07s/it, Training loss=0.252]
Validation epoch5/5: 100%|██████████| 10/10 [00:38<00:00,  3.84s/it]

Epoch: 5, Validation Loss: 0.800694465637207


In [24]:
input_str = "Kidney Failure"

In [25]:
inputs_ads = tokenizer.encode(input_str, return_tensors='pt').to(device)

In [26]:
inputs_ads

tensor([[48374,  1681, 25743]])

In [28]:
outputs = model.generate(inputs_ads,
                         max_length=20,
                         num_return_sequences=1,
                         do_sample=True,
                         top_k=8,
                         temperature=0.7,          # Correct spelling
                         repetition_penalty=1.2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [30]:
outputs

tensor([[48374,  1681, 25743,   930, 36400,   839, 18922,  5072,    11, 11711,
         21545,    11, 18787,    11, 32122,    11, 30874, 50256]])

In [31]:
decode_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [32]:
decode_output

'Kidney Failure | Decreased urine output, fluid retention, fatigue, nausea, vomiting'